In [ ]:
pip install webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# the following two import statements provide the implemetation for Selenium Waits
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import pandas as pd

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)


In [ ]:

house_link = "https://www.politico.com/2022-election/results/"
driver.get(house_link)


In [ ]:

# Get the page source
page_source = driver.page_source

# Parse the page source using Beautiful Soup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table that contains the list of states
container = soup.find('div','styles_columns-container__Kq5vS')

# Extract the list of states from the table
states = [state.get_text(strip=True) for state in container.find_all('a')]

# Print the list of states
print(states)


In [ ]:
states_link = [state.replace(' ', '-') for state in states]

In [ ]:
print (states_link)

In [ ]:
import pandas

df = pd.DataFrame(columns=['Candidate', 'Party', 'Votes', 'Pct%','State', 'District'])

In [ ]:
df

In [ ]:
count=0
final_results = {}

for index, link in enumerate(states_link):
    print(index, 'Loading Data for', states[index])
    result = {}
    driver.get(house_link + str(link.lower())+'/house/')
    try:
        buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[title="Expand"]')))
        buttons = driver.find_elements(By.CSS_SELECTOR, '[title="Expand"]')
        print(' -----',len(buttons), 'button found.')
        for button in buttons:
            button.click() 
            print(states[index], button.text, 'clicked')
        buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[title="Expand"]')))
        buttons = driver.find_elements(By.CSS_SELECTOR, '[title="Expand"]')
        print(' -----',len(buttons), 'button found.')
        for button in buttons:
            button.click() 
            print(states[index], button.text, 'clicked')
    except:
        print(states[index],'no button found')
    # start scraping districts
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #print(soup)

    container = soup.find('div',{'class':'styles_table-container__vTHda'})

    districts = container.find_all('h5')

    districts = [district.get_text(strip=True) for district in districts]


    #scrapping candidate
    sections = []
    sub_list = []
    if container.find_all('div',{'class':'styles_left-title__rNUfI'}):
        sections = container.find_all('div',{'class':'styles_left-title__rNUfI'})
    
    
    if sections != []:
        
        for section_index, section in enumerate(sections):
            candidates = section.find_all('td')
            new_list = []

            list1 = [candidate.get_text(strip=True) for candidate in candidates]
            for item in list1:
                if '(' in item:
                    x = item.split('(')
                    for y in x:
                        new_list.append(y)
            
                elif item =='':
                    new_list.append('0')
                    new_list.append('0%')
                elif item.replace(',','').isdigit() == False and '%' not in item and item!= 'Uncontested' and item !='':
                    new_list.append(item)
                    new_list.append('Other Party')
                elif 'Uncontested' == item:
                    next
                
                else: new_list.append(item)
            
            N = 4
            temp_list = [new_list[n:n+N] for n in range(0,len(new_list), N)]
            for sublist in temp_list:
                sublist.append(states[index])
                sublist.append(districts[section_index])
            sub_list.extend(temp_list)
            

                    
    else: 
        candidates = container.find_all('td')
        list1 = [candidate.get_text(strip=True) for candidate in candidates]
        new_list = []
        for item in list1:
            if '(' in item:
                x = item.split('(')
                for y in x:
                    new_list.append(y)
            elif item =='':
                new_list.append('0')
                new_list.append('0.00%')
            elif item.replace(',','').isdigit() == False and '%' not in item and item!= 'Uncontested' and item !='':
                new_list.append(item)
                new_list.append('Other Party')
            elif 'Uncontested' == item:
                next
                
            else: new_list.append(item)
        
        N=4
        
        sub_list = [new_list[n:n+N] for n in range(0,len(new_list), N)]
        for list in sub_list:
            list.append(states[index])
            list.append('At Large')

    new_df = pd.DataFrame(sub_list, columns = df.columns)
    df = pd.concat([df, new_df])
    

    
    count += 1
    

print(count , 'states scraped')

    

In [ ]:
df

In [ ]:
df = df[df['Candidate'] != 'Hide other candidates']

In [ ]:
incumbent = ['Incumbent' if '*' in candidate else '/' for candidate in df['Candidate']]
df['Incumbent'] = incumbent

In [ ]:
df['Party'] = df['Party'].replace('\)','',regex=True)

In [ ]:
df

In [ ]:
party_dict = {'R': 'Republician', 'D': 'Democratic', 'Ind.': 'Independent Party'}
df['Party'] = df['Party'].replace(party_dict)

In [ ]:
us_state_to_code = {
            "Alabama": "AL",
            "Alaska": "AK",
            "Arizona": "AZ",
            "Arkansas": "AR",
            "California": "CA",
            "Colorado": "CO",
            "Connecticut": "CT",
            "Delaware": "DE",
            "Florida": "FL",
            "Georgia": "GA",
            "Hawaii": "HI",
            "Idaho": "ID",
            "Illinois": "IL",
            "Indiana": "IN",
            "Iowa": "IA",
            "Kansas": "KS",
            "Kentucky": "KY",
            "Louisiana": "LA",
            "Maine": "ME",
            "Maryland": "MD",
            "Massachusetts": "MA",
            "Michigan": "MI",
            "Minnesota": "MN",
            "Mississippi": "MS",
            "Missouri": "MO",
            "Montana": "MT",
            "Nebraska": "NE",
            "Nevada": "NV",
            "New Hampshire": "NH",
            "New Jersey": "NJ",
            "New Mexico": "NM",
            "New York": "NY",
            "North Carolina": "NC",
            "North Dakota": "ND",
            "Ohio": "OH",
            "Oklahoma": "OK",
            "Oregon": "OR",
            "Pennsylvania": "PA",
            "Rhode Island": "RI",
            "South Carolina": "SC",
            "South Dakota": "SD",
            "Tennessee": "TN",
            "Texas": "TX",
            "Utah": "UT",
            "Vermont": "VT",
            "Virginia": "VA",
            "Washington": "WA",
            "West Virginia": "WV",
            "Wisconsin": "WI",
            "Wyoming": "WY",
            "District of Columbia": "DC",
            "American Samoa": "AS",
            "Guam": "GU",
            "Northern Mariana Islands": "MP",
            "Puerto Rico": "PR",
            "United States Minor Outlying Islands": "UM",
            "U.S. Virgin Islands": "VI",
        }

df['State Code'] = df['State'].map(us_state_to_code)
df

In [ ]:
df['Candidate'] = df['Candidate'].replace({'\*': ''}, regex=True)

In [ ]:
df['Votes'] = df['Votes'].replace({',': ''}, regex=True).astype(int)

In [ ]:
df

In [ ]:
df = df.loc[:, ['State', 'State Code', 'District', 'Party', 'Candidate', 'Incumbent', 'Votes', 'Pct%']]

In [ ]:
df['Elected'] = df.groupby(['State', 'District'])['Votes'].transform(max) == df['Votes']

In [ ]:
df

In [ ]:
party = df['Party'].tolist()

In [ ]:
won = []
for index, item in enumerate(party):
    get_index= index
    won.append(party[index])

In [ ]:
len(won)

In [ ]:
df['Won Party'] = won

In [ ]:
df

In [ ]:
df.to_csv('final_house.csv', index=False)